# Feature Engineering
다이소 뷰티 제품 데이터 파생변수 생성

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
products_df = pd.read_parquet('products.parquet')
reviews_df = pd.read_parquet('reviews.parquet')

print("데이터 로드 완료")
print(f"제품: {len(products_df):,}개")
print(f"리뷰: {len(reviews_df):,}건")
print(f"\n제품 컬럼: {products_df.columns.tolist()}")
print(f"\n리뷰 컬럼: {reviews_df.columns.tolist()}")

## A. 상품(Product) 관점: "진짜 인기 상품" 찾기

In [ ]:
# 제품별 리뷰 수 계산
review_counts = reviews_df.groupby('product_code').size().reset_index(name='review_count')

# products_df에 병합
products_df = products_df.merge(review_counts, on='product_code', how='left')
products_df['review_count'] = products_df['review_count'].fillna(0).astype(int)

print(f"리뷰 수 추가 완료")
print(f"리뷰가 있는 제품: {(products_df['review_count'] > 0).sum()}개")
print(f"리뷰가 없는 제품: {(products_df['review_count'] == 0).sum()}개")

In [ ]:
# A-1. Engagement Score (실질적 인기도)
# Score = w1*likes + w2*shares + w3*review_count
w1, w2, w3 = 1.0, 3.0, 2.0  # shares와 reviews에 더 높은 가중치

products_df['engagement_score'] = (
    w1 * products_df['likes'] + 
    w2 * products_df['shares'] + 
    w3 * products_df['review_count']
)

print("[A-1] Engagement Score 생성 완료")
print(f"평균: {products_df['engagement_score'].mean():.2f}")
print(f"최대: {products_df['engagement_score'].max():.2f}")
print("\nTOP 5 인기 제품:")
print(products_df.nlargest(5, 'engagement_score')[['name', 'brand', 'likes', 'shares', 'review_count', 'engagement_score']])

In [ ]:
# A-2. Relative Price Status (카테고리 내 가격 위치)
def categorize_price(row):
    """카테고리 내 가격 백분위수로 분류"""
    if pd.isna(row['price_percentile']):
        return 'Unknown'
    elif row['price_percentile'] >= 75:
        return 'High-end'
    elif row['price_percentile'] >= 25:
        return 'Mid'
    else:
        return 'Budget'

# 카테고리별 가격 백분위수 계산
products_df['price_percentile'] = products_df.groupby('category_2')['price'].rank(pct=True) * 100
products_df['price_status'] = products_df.apply(categorize_price, axis=1)

print("[A-2] Relative Price Status 생성 완료")
print("\n가격대별 분포:")
print(products_df['price_status'].value_counts())
print("\n카테고리별 가격대 분포:")
print(pd.crosstab(products_df['category_2'], products_df['price_status']))

In [ ]:
# A-3. Review Density (리뷰 밀도)
# Review Count / Likes (likes 대비 리뷰 작성 비율)
products_df['review_density'] = products_df.apply(
    lambda x: x['review_count'] / x['likes'] if x['likes'] > 0 else 0,
    axis=1
)

print("[A-3] Review Density 생성 완료")
print(f"평균 리뷰 밀도: {products_df['review_density'].mean():.4f}")
print(f"최대 리뷰 밀도: {products_df['review_density'].max():.4f}")
print("\n리뷰 밀도가 높은 제품 TOP 5 (관여도가 높은 제품):")
print(products_df.nlargest(5, 'review_density')[['name', 'brand', 'likes', 'review_count', 'review_density']])

## B. 리뷰(Review) & 텍스트 관점: "평점 5점의 내막" 파헤치기

In [ ]:
# B-1. Review Length (리뷰 길이)
reviews_df['review_length'] = reviews_df['text'].fillna('').str.len()

print("[B-1] Review Length 생성 완료")
print(f"평균 리뷰 길이: {reviews_df['review_length'].mean():.2f}자")
print(f"중간값: {reviews_df['review_length'].median():.0f}자")
print(f"최대: {reviews_df['review_length'].max()}자")

# 리뷰 길이 구간별 분류
def classify_review_length(length):
    if length < 20:
        return 'Very Short'
    elif length < 50:
        return 'Short'
    elif length < 100:
        return 'Medium'
    else:
        return 'Long'

reviews_df['review_length_category'] = reviews_df['review_length'].apply(classify_review_length)
print("\n리뷰 길이 분포:")
print(reviews_df['review_length_category'].value_counts())

In [ ]:
# B-2. Sentiment Score (감성 점수) - 간단한 키워드 기반
# 긍정/부정 키워드 기반 점수 (실제로는 KoBERT 등 사용 권장)

positive_keywords = ['좋', '최고', '만족', '추천', '굿', '대박', '완벽', '예쁘', '이쁘', '사랑', '훌륭', '감사']
negative_keywords = ['별로', '실망', '안좋', '최악', '아쉽', '후회', '불만', '나쁘', '안맞', '짜증']

def calculate_sentiment_score(text):
    """간단한 키워드 기반 감성 점수 (0~1)"""
    if pd.isna(text) or text == '':
        return 0.5  # 중립
    
    text = str(text)
    positive_count = sum(1 for keyword in positive_keywords if keyword in text)
    negative_count = sum(1 for keyword in negative_keywords if keyword in text)
    
    total = positive_count + negative_count
    if total == 0:
        return 0.5  # 중립
    
    # 0~1 사이로 정규화
    score = (positive_count - negative_count) / (2 * total) + 0.5
    return max(0, min(1, score))

reviews_df['sentiment_score'] = reviews_df['text'].apply(calculate_sentiment_score)

print("[B-2] Sentiment Score 생성 완료")
print(f"평균 감성 점수: {reviews_df['sentiment_score'].mean():.3f}")
print(f"중간값: {reviews_df['sentiment_score'].median():.3f}")
print("\n감성 점수 분포:")
print(reviews_df['sentiment_score'].describe())

In [ ]:
# B-3. 키워드 기반 속성 태그
keywords = {
    'has_delivery': ['배송', '배달', '택배', '빠르게'],
    'has_price': ['가성비', '가격', '저렴', '비싸', '싸'],
    'has_quality': ['재질', '품질', '질감', '퀄리티'],
    'has_gift': ['선물', '선물용', '증정'],
    'has_repurchase': ['재구매', '또 사', '또 구매', '계속 사']
}

for tag, words in keywords.items():
    reviews_df[tag] = reviews_df['text'].fillna('').apply(
        lambda x: 1 if any(word in str(x) for word in words) else 0
    )

print("[B-3] 키워드 기반 속성 태그 생성 완료")
print("\n키워드 언급 비율:")
for tag in keywords.keys():
    count = reviews_df[tag].sum()
    pct = count / len(reviews_df) * 100
    print(f"{tag}: {count:,}건 ({pct:.2f}%)")

## C. 유저(User) 관점: "누가 진성 고객인가?"

In [ ]:
# C-1. User Activity Level (유저 활동 등급)
user_review_counts = reviews_df.groupby('user_id').size().reset_index(name='user_total_reviews')
reviews_df = reviews_df.merge(user_review_counts, on='user_id', how='left')

def classify_user_activity(count):
    if count <= 1:
        return 'Newbie'
    elif count <= 5:
        return 'Junior'
    elif count <= 20:
        return 'Regular'
    else:
        return 'VIP'

reviews_df['user_activity_level'] = reviews_df['user_total_reviews'].apply(classify_user_activity)

print("[C-1] User Activity Level 생성 완료")
print("\n유저 등급 분포:")
print(reviews_df['user_activity_level'].value_counts())
print(f"\n고유 유저 수: {reviews_df['user_id'].nunique():,}명")

In [ ]:
# C-2. User Average Rating (유저 평균 평점)
user_avg_rating = reviews_df.groupby('user_id')['rating'].mean().reset_index(name='user_avg_rating')
reviews_df = reviews_df.merge(user_avg_rating, on='user_id', how='left')

# 평점 부여 성향 분류
def classify_rating_tendency(avg_rating):
    if avg_rating >= 4.8:
        return 'Always Positive'
    elif avg_rating >= 4.0:
        return 'Mostly Positive'
    elif avg_rating >= 3.0:
        return 'Balanced'
    else:
        return 'Critical'

reviews_df['user_rating_tendency'] = reviews_df['user_avg_rating'].apply(classify_rating_tendency)

print("[C-2] User Average Rating 생성 완료")
print(f"평균 유저 평점: {reviews_df['user_avg_rating'].mean():.3f}")
print("\n유저 평점 성향 분포:")
print(reviews_df['user_rating_tendency'].value_counts())

In [ ]:
# C-3. Repurchase Indicator (재구매 추정)
# 같은 유저가 같은 브랜드 또는 같은 카테고리 제품을 다시 구매했는지

# 제품 정보 병합 (brand, category 정보 가져오기)
reviews_with_product = reviews_df.merge(
    products_df[['product_code', 'brand', 'category_2']], 
    on='product_code', 
    how='left'
)

# 날짜 변환
reviews_with_product['date'] = pd.to_datetime(reviews_with_product['date'])

# 유저별로 정렬
reviews_with_product = reviews_with_product.sort_values(['user_id', 'date'])

# 같은 브랜드 재구매 여부
reviews_with_product['is_brand_repurchase'] = (
    reviews_with_product.groupby('user_id')['brand']
    .transform(lambda x: x.duplicated(keep=False))
).astype(int)

# 같은 카테고리 재구매 여부
reviews_with_product['is_category_repurchase'] = (
    reviews_with_product.groupby('user_id')['category_2']
    .transform(lambda x: x.duplicated(keep=False))
).astype(int)

# 원본 reviews_df에 병합
reviews_df['is_brand_repurchase'] = reviews_with_product['is_brand_repurchase']
reviews_df['is_category_repurchase'] = reviews_with_product['is_category_repurchase']

print("[C-3] Repurchase Indicator 생성 완료")
print(f"브랜드 재구매: {reviews_df['is_brand_repurchase'].sum():,}건 ({reviews_df['is_brand_repurchase'].sum()/len(reviews_df)*100:.2f}%)")
print(f"카테고리 재구매: {reviews_df['is_category_repurchase'].sum():,}건 ({reviews_df['is_category_repurchase'].sum()/len(reviews_df)*100:.2f}%)")

## D. 시계열(Time) 관점

In [ ]:
# D-1. Seasonality (계절성)
reviews_df['date'] = pd.to_datetime(reviews_df['date'])
reviews_df['year'] = reviews_df['date'].dt.year
reviews_df['month'] = reviews_df['date'].dt.month
reviews_df['day_of_week'] = reviews_df['date'].dt.dayofweek  # 0=월요일, 6=일요일
reviews_df['day_name'] = reviews_df['date'].dt.day_name()

# 계절 분류
def classify_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

reviews_df['season'] = reviews_df['month'].apply(classify_season)

print("[D-1] Seasonality 생성 완료")
print("\n월별 리뷰 수:")
print(reviews_df['month'].value_counts().sort_index())
print("\n계절별 리뷰 수:")
print(reviews_df['season'].value_counts())
print("\n요일별 리뷰 수:")
print(reviews_df['day_name'].value_counts())

In [ ]:
# D-2. Days Since First Review (출시 후 경과일)
# 제품별 첫 리뷰 날짜
first_review_dates = reviews_df.groupby('product_code')['date'].min().reset_index(name='first_review_date')
reviews_df = reviews_df.merge(first_review_dates, on='product_code', how='left')

# 경과일 계산
reviews_df['days_since_first_review'] = (
    reviews_df['date'] - reviews_df['first_review_date']
).dt.days

# 신상품 여부 (첫 리뷰 후 30일 이내)
reviews_df['is_new_product'] = (reviews_df['days_since_first_review'] <= 30).astype(int)

print("[D-2] Days Since First Review 생성 완료")
print(f"평균 경과일: {reviews_df['days_since_first_review'].mean():.1f}일")
print(f"최대 경과일: {reviews_df['days_since_first_review'].max()}일")
print(f"\n신상품(30일 이내) 리뷰: {reviews_df['is_new_product'].sum():,}건 ({reviews_df['is_new_product'].sum()/len(reviews_df)*100:.2f}%)")

## 최종 데이터 저장

In [ ]:
# Parquet으로 저장
products_df.to_parquet('products_with_features.parquet', index=False)
reviews_df.to_parquet('reviews_with_features.parquet', index=False)

print("="*80)
print("Feature Engineering 완료!")
print("="*80)
print(f"\nproducts_with_features.parquet 저장 완료")
print(f"  - 제품 수: {len(products_df):,}개")
print(f"  - 컬럼 수: {len(products_df.columns)}개")
print(f"  - 추가된 컬럼: review_count, engagement_score, price_percentile, price_status, review_density")

print(f"\nreviews_with_features.parquet 저장 완료")
print(f"  - 리뷰 수: {len(reviews_df):,}건")
print(f"  - 컬럼 수: {len(reviews_df.columns)}개")
print(f"  - 추가된 컬럼: review_length, sentiment_score, 키워드 태그들, user_activity_level, user_avg_rating, 재구매 지표, 시계열 변수들")

print("\n생성된 파일:")
print("  - products_with_features.parquet")
print("  - reviews_with_features.parquet")

In [ ]:
# 최종 컬럼 목록 확인
print("\n=" * 80)
print("최종 컬럼 목록")
print("=" * 80)

print("\n[Products DataFrame]")
for i, col in enumerate(products_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n[Reviews DataFrame]")
for i, col in enumerate(reviews_df.columns, 1):
    print(f"{i:2d}. {col}")